In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import torch
import dimod
import dwave_networkx as dnx

In [2]:
#return edges of networkx graph as a list
def nxgetEdges(g):
    edges = []
    for u,v in g.edges():
        if 'weight' in g[u][v]:
            edges.append([u,v,g[u][v]['weight']])
        else: edges.append([u,v])
    return edges

## Download Data  from https://networks.skewed.de/

### 1) Aarhus: https://networks.skewed.de/net/cs_department/files/cs_department.csv.zip

### 2) Jazz: https://networks.skewed.de/net/jazz_collab/files/jazz_collab.csv.zip

### 3) Facebook: https://networks.skewed.de/net/facebook_friends/files/facebook_friends.csv.zip

### 4) School: https://networks.skewed.de/net/spanish_highschools/files/11_2.csv.zip

In [3]:
def make_dataset(_name = 'aarhus', n = 61):
    print(f'Making {_name} Graph...')
    df = pd.read_csv(f'./datasets/{_name}/edges.csv')
    Gs = [nx.Graph() for _ in range(3)] #0->Two disconnected components | 1->bipartite | 2->whole graph
    for i in range(3):
        for j in range(n):
            Gs[i].add_node(j)
            Gs[i].add_edge(j, j)
    alledges = 0
    for index, row in df.iterrows():
        u = row['# source']
        v = row[' target']
        Gs[2].add_edge(u, v)
        alledges += 1
    
    #
    
    sampler = dimod.SimulatedAnnealingSampler()
    cut = dnx.maximum_cut(Gs[2], sampler)
    
    print('n ',n,' and ',' m ', alledges)
    print(f'Nodes of Layer 1 is {len(cut)}')
    print(f'Nodes of Layer 2 is {n - len(cut)}')
    
    edges1 = 0
    edges2 = 0
    edges3 = 0
    for index, row in df.iterrows():
        u = row['# source']
        v = row[' target']
        if (u in cut) and (v in cut):  
            Gs[0].add_edge(u, v)
            edges1 += 1
        elif (u not in cut) and (v not in cut):  
            Gs[0].add_edge(u, v)
            edges2 += 1
        else: 
            Gs[1].add_edge(u, v)
            edges3 += 1
            
    print(f'Edges of Layer 1 is {edges1}')
    print(f'Edges of Layer 2 is {edges2}')
    print(f'Edges of Layer 3 is {edges3}')
    print()
    
    Ws = []
    for i in range(3):
        A = nx.adjacency_matrix(Gs[i]).todense()
        Ws.append(torch.from_numpy((A.T*1/A.sum(axis = 1).T).T).double())
    return Ws
#('aarhus', 61), ('jazz', 198), ('facebook', 362), ('school', 702)
datasets_n = [('school', 702)]
for _name_n in datasets_n:
    make_dataset(_name = _name_n[0], n = _name_n[1])

Making school Graph...
n  702  and   m  11960
Nodes of Layer 1 is 398
Nodes of Layer 2 is 304
Edges of Layer 1 is 2276
Edges of Layer 2 is 2533
Edges of Layer 3 is 7151



##### Number of nodes and edges may differ as the maximum-cut algorithm does not provide an optimal solution